In [ ]:
%env OPENAI_API_KEY=<api key here>

In [18]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

from kagent.tools import BuiltInTool

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks that can be executed by the team members. DO NOT MAKE UP ADDITIONAL AND UNNECESSARY SUBTASKS.
    Your team members are:
        istio_agent: Performs Istio resource generation tasks.
        k8s_agent: Performs Kubernetes tasks.

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

k8s_agent = AssistantAgent(
    "k8s_agent",
    description="An agent for k8s operations",
    tools=[BuiltInTool("k8s.apply_manifest")],
    model_client=model_client,
    system_message="""
    You are a k8s agent. You know how to interact with a Kubernetes cluster.
    """,
)

istio_agent = AssistantAgent(
    "istio_agent",
    description="An agent for creating Istio resources",
    tools=[BuiltInTool("istio.generate_resource")],
    model_client=model_client,
    system_message="""
    You are an Istio agent that knows how to generate YAML for Istio resources.
    You only respond with YAML for Istio resources and nothing else. If you don't know the answer, say "I don't know".

    The resources you can generate are:
    - Gateway
    - VirtualService
    - AuthorizationPolicy
    - PeerAuthentication
    """,
)

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination


team = SelectorGroupChat(
    [planning_agent, istio_agent, k8s_agent],
    model_client=model_client,
    termination_condition=termination,
    allow_repeated_speaker=True,
)

task = "Expose the productpage through the ingress gateway and create an authorizationpolicy that denies all traffic to the productpage service"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))


---------- user ----------
Expose the productpage through the ingress gateway and create an authorizationpolicy that denies all traffic to the productpage service
---------- PlanningAgent ----------
To achieve this, we need to perform the following subtasks:

1. Configure the Istio Ingress Gateway to route external traffic to the productpage service.
2. Create a Kubernetes Service for the productpage if it's not already defined.
3. Define an AuthorizationPolicy to deny all traffic to the productpage service.

Here are the specific tasks assigned to the agents:

1. istio_agent: Create a VirtualService to expose the productpage through the Istio Ingress Gateway.
2. k8s_agent: Verify and ensure the Kubernetes Service for productpage is correctly defined.
3. istio_agent: Create an AuthorizationPolicy in the namespace of the service to deny all traffic to the productpage.

Please proceed with these tasks.
---------- istio_agent ----------
[FunctionCall(id='call_khZ5etLWaIMBFvKjPywdBBwP', ar

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Expose the productpage through the ingress gateway and create an authorizationpolicy that denies all traffic to the productpage service', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=164, completion_tokens=149), content="To achieve this, we need to perform the following subtasks:\n\n1. Configure the Istio Ingress Gateway to route external traffic to the productpage service.\n2. Create a Kubernetes Service for the productpage if it's not already defined.\n3. Define an AuthorizationPolicy to deny all traffic to the productpage service.\n\nHere are the specific tasks assigned to the agents:\n\n1. istio_agent: Create a VirtualService to expose the productpage through the Istio Ingress Gateway.\n2. k8s_agent: Verify and ensure the Kubernetes Service for productpage is correctly defined.\n3. istio_agent: Create an AuthorizationPolicy in the namespace of the service 

In [ ]:
print(team.dump_component().model_dump_json(indent=2))